In [10]:
import re
import datetime
import pandas as pd
import numpy as np
import requests
import io
import json
import random
from random import randint
from base64 import b64encode, b64decode
from django.contrib.auth.hashers import make_password
from django.core.files.base import ContentFile
from decimal import Decimal
try:
    from StringIO import StringIO
except ImportError:
    from io import StringIO
    from io import BytesIO
google_sheet_url = 'https://docs.google.com/spreadsheets/d/1HBTh1UQtSpxRIeV8Xhvg303z8nijbhjMO4NvleMT5L0/export?format=csv'

r = requests.get(google_sheet_url)
data = r.content
df = pd.read_csv(BytesIO(data))
local_index = []
df.head(5)

,UNIQUE CODE,SUPERVISOR NAME,SUPERVISOR USERNAME,UNIQUE CODE.1,AGENT NAME,AGENT USERNAME,UNIQUE CODE TO FARMERS,CLUSTERS,CLUSTER_ID,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,1.0,AJITH S,ajith,1.0,ARASA SETTY,8217357218,GA 01/01,Berambadi,1,NaN,NaN,NaN,NaN
1,1.0,AJITH S,ajith,2.0,BANGARANAIKA,9980310639,GA 01/02,Berambadi,1,NaN,NaN,NaN,NaN
2,1.0,AJITH S,ajith,3.0,MAHENDRA,ccgb_kollegala,GA 01/03,Berambadi,1,NaN,NaN,NaN,NaN
3,1.0,AJITH S,ajith,4.0,MADAPPA G B,9108390550,GA 01/04,Berambadi,1,NaN,NaN,NaN,NaN
4,1.0,AJITH S,ajith,5.0,MADAPPA,8548092445,GA 01/05,Berambadi,1,NaN,NaN,NaN,NaN


In [2]:
# --------------------------------------------
# to know the unmapped cluster farmers
farmers = Farmer.objects.all().order_by('id')
unclustered_farmer_ids = []
agent_unmaped_ids = []
for farmer in farmers:
    if not FarmerClusterSeasonMap.objects.filter(farmer_id=farmer.id).exists():
        unclustered_farmer_ids.append(farmer.id)
unclustered_farmer_ids.append(2639)
# unclustered_farmer_ids

In [21]:
for farmer in Farmer.objects.all().exclude(id__in=unclustered_farmer_ids).order_by('id'):
    print(farmer.id)
    cluster_id = FarmerClusterSeasonMap.objects.filter(farmer_id=farmer.id)[0].cluster.id
    mapping_id = FarmerClusterSeasonMap.objects.filter(farmer_id=farmer.id)[0].id
    season_id = FarmerClusterSeasonMap.objects.filter(farmer_id=farmer.id)[0].season_id
    agent_id = AgentFarmerMap.objects.get(farmer_id=mapping_id).agent.id
    agent_name = AgentFarmerMap.objects.get(farmer_id=mapping_id).agent.username
    if agent_name == '9731636749' or agent_name == '9844840856' or agent_name == '9944318848' or agent_name == '7353104238':
        continue
    print('searching agent first_name:', AgentFarmerMap.objects.get(farmer_id=mapping_id).agent.first_name)
    print('searching agent last_name:', AgentFarmerMap.objects.get(farmer_id=mapping_id).agent.last_name)
    print('searching agent user_name:', AgentFarmerMap.objects.get(farmer_id=mapping_id).agent.username)
    
    officer_name = df[df['AGENT USERNAME']== agent_name]['SUPERVISOR USERNAME'].iloc[0]
    officer_id = User.objects.get(username=officer_name).id
    
    print(farmer.id)
    print('cluster_id :', cluster_id)
    print('map_id: ', mapping_id)
    print('officer_id :', officer_id)
    print('agent_id :', agent_id)
    code = generate_farmer_code(cluster_id, agent_id, officer_id,season_id)
    Farmer.objects.filter(id=farmer.id).update(code=code)
    print('code :', code)
    print('-----------------')

1
searching agent first_name: Arasashetty
searching agent last_name: Kongalashetty
searching agent user_name: 8217357218
1
cluster_id : 8
map_id:  224
officer_id : 22
agent_id : 31
19NAA
code : 19NAA001
-----------------
2
searching agent first_name: Arasashetty
searching agent last_name: Kongalashetty
searching agent user_name: 8217357218
2
cluster_id : 8
map_id:  225
officer_id : 22
agent_id : 31
19NAA
code : 19NAA002
-----------------
3
searching agent first_name: Arasashetty
searching agent last_name: Kongalashetty
searching agent user_name: 8217357218
3
cluster_id : 8
map_id:  226
officer_id : 22
agent_id : 31
19NAA
code : 19NAA003
-----------------
4
searching agent first_name: Arasashetty
searching agent last_name: Kongalashetty
searching agent user_name: 8217357218
4
cluster_id : 8
map_id:  227
officer_id : 22
agent_id : 31
19NAA
code : 19NAA004
-----------------
5
searching agent first_name: Arasashetty
searching agent last_name: Kongalashetty
searching agent user_name: 821735

searching agent first_name: Bangaranaika
searching agent last_name: Siddanaika
searching agent user_name: 9980310639
48
cluster_id : 8
map_id:  343
officer_id : 22
agent_id : 32
19NAB
code : 19NAB015
-----------------
49
searching agent first_name: Bangaranaika
searching agent last_name: Siddanaika
searching agent user_name: 9980310639
49
cluster_id : 8
map_id:  344
officer_id : 22
agent_id : 32
19NAB
code : 19NAB016
-----------------
50
searching agent first_name: Bangaranaika
searching agent last_name: Siddanaika
searching agent user_name: 9980310639
50
cluster_id : 8
map_id:  345
officer_id : 22
agent_id : 32
19NAB
code : 19NAB017
-----------------
51
searching agent first_name: Bangaranaika
searching agent last_name: Siddanaika
searching agent user_name: 9980310639
51
cluster_id : 8
map_id:  346
officer_id : 22
agent_id : 32
19NAB
code : 19NAB018
-----------------
52
searching agent first_name: Bangaranaika
searching agent last_name: Siddanaika
searching agent user_name: 9980310639

searching agent first_name: Madappa
searching agent last_name: Gurappa
searching agent user_name: 8548092445
96
cluster_id : 8
map_id:  502
officer_id : 22
agent_id : 35
19NAM
code : 19NAM012
-----------------
97
searching agent first_name: Madappa
searching agent last_name: Gurappa
searching agent user_name: 8548092445
97
cluster_id : 8
map_id:  503
officer_id : 22
agent_id : 35
19NAM
code : 19NAM013
-----------------
98
searching agent first_name: Madappa
searching agent last_name: Gurappa
searching agent user_name: 8548092445
98
cluster_id : 8
map_id:  504
officer_id : 22
agent_id : 35
19NAM
code : 19NAM014
-----------------
99
searching agent first_name: Madappa
searching agent last_name: Gurappa
searching agent user_name: 8548092445
99
cluster_id : 8
map_id:  505
officer_id : 22
agent_id : 35
19NAM
code : 19NAM015
-----------------
100
searching agent first_name: Madappa
searching agent last_name: Gurappa
searching agent user_name: 8548092445
100
cluster_id : 8
map_id:  506
office

19NAM
code : 19NAM059
-----------------
144
searching agent first_name: Madappa G B
searching agent last_name: Bellappa
searching agent user_name: 9108390550
144
cluster_id : 8
map_id:  559
officer_id : 22
agent_id : 36
19NAM
code : 19NAM060
-----------------
145
searching agent first_name: Madappa G B
searching agent last_name: Bellappa
searching agent user_name: 9108390550
145
cluster_id : 8
map_id:  560
officer_id : 22
agent_id : 36
19NAM
code : 19NAM061
-----------------
146
searching agent first_name: Madappa G B
searching agent last_name: Bellappa
searching agent user_name: 9108390550
146
cluster_id : 8
map_id:  561
officer_id : 22
agent_id : 36
19NAM
code : 19NAM062
-----------------
147
searching agent first_name: Madappa G B
searching agent last_name: Bellappa
searching agent user_name: 9108390550
147
cluster_id : 8
map_id:  562
officer_id : 22
agent_id : 36
19NAM
code : 19NAM063
-----------------
148
searching agent first_name: Madappa G B
searching agent last_name: Bellappa


searching agent first_name: Madappa G B
searching agent last_name: Bellappa
searching agent user_name: 9108390550
191
cluster_id : 8
map_id:  602
officer_id : 22
agent_id : 36
19NAM
code : 19NAM107
-----------------
192
searching agent first_name: Madappa G B
searching agent last_name: Bellappa
searching agent user_name: 9108390550
192
cluster_id : 8
map_id:  603
officer_id : 22
agent_id : 36
19NAM
code : 19NAM108
-----------------
193
searching agent first_name: Madappa G B
searching agent last_name: Bellappa
searching agent user_name: 9108390550
193
cluster_id : 8
map_id:  604
officer_id : 22
agent_id : 36
19NAM
code : 19NAM109
-----------------
194
searching agent first_name: Madappa G B
searching agent last_name: Bellappa
searching agent user_name: 9108390550
194
cluster_id : 8
map_id:  605
officer_id : 22
agent_id : 36
19NAM
code : 19NAM110
-----------------
195
searching agent first_name: Madappa G B
searching agent last_name: Bellappa
searching agent user_name: 9108390550
195
cl

...

code : 20BAM143
-----------------
5012
searching agent first_name: Madappa
searching agent last_name: Gurappa
searching agent user_name: 8548092445
5012
cluster_id : 1
map_id:  5009
officer_id : 22
agent_id : 35
20BAM
code : 20BAM144
-----------------
5013
searching agent first_name: Madappa
searching agent last_name: Gurappa
searching agent user_name: 8548092445
5013
cluster_id : 1
map_id:  5010
officer_id : 22
agent_id : 35
20BAM
code : 20BAM145
-----------------
5014
searching agent first_name: Madappa
searching agent last_name: Gurappa
searching agent user_name: 8548092445
5014
cluster_id : 1
map_id:  5011
officer_id : 22
agent_id : 35
20BAM
code : 20BAM146
-----------------
5015
searching agent first_name: Madappa
searching agent last_name: Gurappa
searching agent user_name: 8548092445
5015
cluster_id : 1
map_id:  5012
officer_id : 22
agent_id : 35
20BAM
code : 20BAM147
-----------------
5016
searching agent first_name: Madappa
searching agent last_name: Gurappa
searching agent us

searching agent first_name: Mahesh
searching agent last_name: Madappa
searching agent user_name: 9008791197
5057
cluster_id : 1
map_id:  5054
officer_id : 27
agent_id : 74
20BPM
code : 20BPM096
-----------------
5058
searching agent first_name: Mahesh
searching agent last_name: Madappa
searching agent user_name: 9008791197
5058
cluster_id : 3
map_id:  5055
officer_id : 27
agent_id : 74
20BPM
code : 20BPM097
-----------------
5059
searching agent first_name: Mahesh
searching agent last_name: Madappa
searching agent user_name: 9008791197
5059
cluster_id : 3
map_id:  5056
officer_id : 27
agent_id : 74
20BPM
code : 20BPM098
-----------------
5060
searching agent first_name: Mahesha 
searching agent last_name: Chikkarajappa
searching agent user_name: 9964757811
5060
cluster_id : 3
map_id:  5057
officer_id : 27
agent_id : 51
20BPM
code : 20BPM099
-----------------
5061
searching agent first_name: Mahesha 
searching agent last_name: Chikkarajappa
searching agent user_name: 9964757811
5061
clu

5104
cluster_id : 4
map_id:  5101
officer_id : 27
agent_id : 49
20BPC
code : 20BPC101
-----------------
5105
searching agent first_name: Chandrappa
searching agent last_name: Javanappa
searching agent user_name: 9741650872
5105
cluster_id : 4
map_id:  5102
officer_id : 27
agent_id : 49
20BPC
code : 20BPC102
-----------------
5106
searching agent first_name: Chandrappa
searching agent last_name: Javanappa
searching agent user_name: 9741650872
5106
cluster_id : 4
map_id:  5103
officer_id : 27
agent_id : 49
20BPC
code : 20BPC103
-----------------
5107
searching agent first_name: Chandrappa
searching agent last_name: Javanappa
searching agent user_name: 9741650872
5107
cluster_id : 4
map_id:  5104
officer_id : 27
agent_id : 49
20BPC
code : 20BPC104
-----------------
5108
searching agent first_name: Chandrappa
searching agent last_name: Javanappa
searching agent user_name: 9741650872
5108
cluster_id : 4
map_id:  5105
officer_id : 27
agent_id : 49
20BPC
code : 20BPC105
-----------------
5109

searching agent first_name: Parameswaran_sargur
searching agent last_name: Madashetty
searching agent user_name: 9738080982
5151
cluster_id : 6
map_id:  5148
officer_id : 24
agent_id : 63
20SPP
code : 20SPP060
-----------------
5152
searching agent first_name: Kemparaju_Beerambadi
searching agent last_name: Madappa
searching agent user_name: 01234512347
5152
cluster_id : 1
map_id:  5149
officer_id : 22
agent_id : 121
20BAK
code : 20BAK055
-----------------
5153
searching agent first_name: Kemparaju_Beerambadi
searching agent last_name: Madappa
searching agent user_name: 01234512347
5153
cluster_id : 1
map_id:  5150
officer_id : 22
agent_id : 121
20BAK
code : 20BAK056
-----------------
5154
searching agent first_name: Kemparaju_Beerambadi
searching agent last_name: Madappa
searching agent user_name: 01234512347
5154
cluster_id : 1
map_id:  5151
officer_id : 22
agent_id : 121
20BAK
code : 20BAK057
-----------------
5155
searching agent first_name: Kemparaju_Beerambadi
searching agent las

searching agent last_name: Muddappa
searching agent user_name: ccgb_sargur_ch_nagar
5187
cluster_id : 6
map_id:  5184
officer_id : 28
agent_id : 91
20SSP
code : 20SSP009
-----------------
5188
searching agent first_name: Puttamallappa
searching agent last_name: Muddappa
searching agent user_name: ccgb_sargur_ch_nagar
5188
cluster_id : 6
map_id:  5185
officer_id : 28
agent_id : 91
20SSP
code : 20SSP010
-----------------
5189
searching agent first_name: Puttamallappa
searching agent last_name: Muddappa
searching agent user_name: ccgb_sargur_ch_nagar
5189
cluster_id : 6
map_id:  5186
officer_id : 28
agent_id : 91
20SSP
code : 20SSP011
-----------------
5190
searching agent first_name: Puttamallappa
searching agent last_name: Muddappa
searching agent user_name: ccgb_sargur_ch_nagar
5190
cluster_id : 6
map_id:  5187
officer_id : 28
agent_id : 91
20SSP
code : 20SSP012
-----------------
5191
searching agent first_name: Puttamallappa
searching agent last_name: Muddappa
searching agent user_nam

In [20]:
def generate_farmer_code(cluster_id, agent_id, officer_id,season_id):
    season = Season.objects.get(id=season_id).year
    season_name = str(season.strftime('%y'))
    cluster_name = Cluster.objects.get(id=cluster_id).name.upper()
    agent_name = User.objects.get(id=agent_id).first_name.upper()
    officer_name = User.objects.get(id=officer_id).username.upper()
    prefix = season_name + cluster_name[0]  + officer_name[0] + agent_name[0]
    print(prefix)
    if IdBank.objects.filter(purpose='farmer', prefix_code=prefix).exists():
        last_code = IdBank.objects.get(purpose='farmer', prefix_code=prefix).last_count
        IdBank.objects.filter(purpose='farmer', prefix_code=prefix).update(
            last_count = last_code + 1
        )
    else:
        last_code = 0
        id_bank_obj = IdBank(
            purpose = 'farmer',
            prefix_code = prefix,
            last_count = 1,
            business_id=1)
        id_bank_obj.save()
    generated_code_number = last_code + 1
    suffix = str(generated_code_number).zfill(3)
    code = str(prefix) +suffix
    # print(code)
    return code

In [19]:
idb =1 
for farmer in Farmer.objects.all().exclude(id__in=unclustered_farmer_ids).order_by('id'):
    Farmer.objects.filter(id=farmer.id).update(code=idb)
    idb += 1

'19'

In [23]:
for index, row in df.iterrows():
    print('----------------------------------')
    print('agent_name :', row['AGENT NAME'])
    print('cluster_name :', row['CLUSTERS'])
    print('codes', list(AgentFarmerMap.objects.filter(agent__username=row['AGENT USERNAME'], farmer__cluster_id=row['CLUSTER_ID']).order_by('farmer__farmer__code').values_list('farmer__farmer__code', flat=True)))


----------------------------------
agent_name : ARASA SETTY
cluster_name : Berambadi
codes ['20BAA001', '20BAA002', '20BAA003', '20BAA004', '20BAA005', '20BAA006', '20BAA007', '20BAA008', '20BAA009', '20BAA010', '20BAA011', '20BAA012', '20BAA013', '20BAA014', '20BAA015', '20BAA016', '20BAA017', '20BAA018', '20BAA019', '20BAA020', '20BAA021', '20BAA022', '20BAA023', '20BAA024', '20BAA025', '20BAA026', '20BAA027', '20BAA028', '20BAA029']
----------------------------------
agent_name : BANGARANAIKA
cluster_name : Berambadi
codes ['20BAB001', '20BAB038', '20BAB039', '20BAB040', '20BAB041', '20BAB042', '20BAB043', '20BAB044', '20BAB045', '20BAB046', '20BAB047', '20BAB048', '20BAB049', '20BAB050', '20BAB051', '20BAB052', '20BAB053', '20BAB054', '20BAB055', '20BAB056', '20BAB057', '20BAB058', '20BAB059', '20BAB060', '20BAB061', '20BAB062', '20BAB063', '20BAB064', '20BAB065', '20BAB066', '20BAB067', '20BAB068', '20BAB069', '20BAB070', '20BAB071', '20BAB072', '20BAB073', '20BAB074', '20BAB075',

ValueError: cannot convert float NaN to integer